In [1]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import numpy.matlib
%matplotlib

Using matplotlib backend: MacOSX


In [2]:
file = open('function_words.csv','r')
tmp = file.read()
fwords = tmp.split('\n')
fwords = fwords[:-1]
# Create a helper dictionary
fwords_dict = {}
for idx, word in enumerate(fwords):
    fwords_dict[word] = idx+1

In [3]:
def functionWordWAN(sentence, fwords_dict, window_size, alpha):
    wan = np.zeros((len(fwords_dict), len(fwords_dict)))
    for idx, pivot in enumerate(sentence[:-1]): # don't include last word as pivot
        if (not fwords_dict.get(pivot)):
            continue
        else:
            sentence_slice = sentence[idx:(idx+window_size)]
            for it, word in enumerate(sentence_slice[1:]): # don't include pivot word
                if (fwords_dict.get(word)):
                    if (fwords_dict.get(word) != fwords_dict.get(pivot)): # no self loops
                        r = fwords_dict.get(pivot)
                        c = fwords_dict.get(word)
                        wan[r-1,c-1] += pow(alpha,it+1)
                           
    return wan

In [4]:
def normalizeWAN(raw_wan):
    sums = raw_wan.sum(axis=1)
    sums = np.matlib.repmat(sums,len(fwords),1)
    sums = sums.T
    norm_wan = raw_wan / sums
    norm_wan = np.nan_to_num(norm_wan) # Make sure nans from zero division are zeros
    return norm_wan

In [5]:
wan = functionWordEntropy(fwords, fwords_dict, 10, 0.75)

In [6]:
# normalize WAN rows to represent as markov chain
norm_wan = normalizeWan(wan)
# Optional visualize as result of test
# plt.imshow(norm_out)
# plt.show()

# Optional sum across columns (will show all 1's except 0 in last position)
# norm_wan.sum(axis=1)

## A simple Test

In [7]:
alpha = 0.75
sentence = ['the','cat','in','the','hat','bought','a','baseball','bat','i','am','happy']
window_size = 6

In [8]:
wan2 = functionWordEntropy(sentence, fwords_dict, 10, alpha)

In [9]:
norm_wan2 = normalizeWan(wan2)

In [10]:
plt.imshow(norm_wan2)
plt.show()